In [41]:
%load_ext autoreload
%autoreload 2

NameError: name 'fix' is not defined

In [42]:
import torchvision, torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, TensorDataset, DataLoader
import numpy as np, sklearn
import torch.nn.functional as F
import os
import requests
import zipfile
import io
from shutil import copyfile
from resnet import ResNet10
from torchvision.transforms.transforms import CenterCrop, Resize
import torch.optim as optim


gds = lambda dataset, cfg: torch.utils.data.DataLoader(
    TensorDataset(*[torch.from_numpy(x) for x in dataset]), batch_size=cfg["batchSize"]
)


def getnorm(dname):
    if dname == "Ci10":
        return (
            torch.from_numpy(
                np.array((0.4914, 0.4822, 0.4465), np.float32).reshape(1, 3, 1, 1)
            ).cuda(),
            torch.from_numpy(
                np.array((0.2023, 0.1994, 0.2010), np.float32).reshape(1, 3, 1, 1)
            ).cuda(),
        )
    elif dname == "Ci100":
        return (
            torch.from_numpy(
                np.array((0.5060725, 0.48667726, 0.4421305), np.float32).reshape(
                    1, 3, 1, 1
                )
            ).cuda(),
            torch.from_numpy(
                np.array((0.2675421, 0.25593522, 0.27593908), np.float32).reshape(
                    1, 3, 1, 1
                )
            ).cuda(),
        )
    elif dname == "Fash":
        return (
            torch.from_numpy(np.array((0.281), np.float32).reshape(1, 1, 1, 1)).cuda(),
            torch.from_numpy(np.array((0.352), np.float32).reshape(1, 1, 1, 1)).cuda(),
        )
    elif dname == "MNIST":
        return (
            torch.from_numpy(np.array((0.1307), np.float32).reshape(1, 1, 1, 1)).cuda(),
            torch.from_numpy(np.array((0.3081), np.float32).reshape(1, 1, 1, 1)).cuda(),
        )
    elif dname == "TinyImgNet":
        return (
            torch.from_numpy(
                np.array((0.4802, 0.4481, 0.3975), np.float32).reshape(3, 1, 1)
            ),
            torch.from_numpy(
                np.array((0.2302, 0.2265, 0.2262), np.float32).reshape(3, 1, 1)
            ),
        )


def getFullDS(cfg, reconstr=False):
    dname = cfg["ds"][0]
    trans = transforms.Compose([transforms.ToTensor()])
    refu = lambda x: F.interpolate(x.unsqueeze(0), size=64).squeeze(0)
    if dname == "Ci10":
        cdat = (
            torchvision.datasets.CIFAR10
        )  # transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) #transform = transforms.Compose([transforms.ToTensor(), norm])
        cfg["imCh"] = 3
    elif dname == "Ci100":
        cdat = (
            torchvision.datasets.CIFAR100
        )  # transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        cfg["imCh"] = 3
    elif dname == "Fash":
        cdat = torchvision.datasets.FashionMNIST
        # img = img - np.array([0.281])            img = img / np.array([0.352])
        trans = transforms.Compose([transforms.ToTensor(), refu])
        cfg["imCh"] = 1
    elif dname == "MNIST":
        cdat = torchvision.datasets.MNIST
        # trans = transforms.Compose([transforms.ToTensor(), refu, transforms.Normalize(0.1307,0.3081)])
        trans = transforms.Compose([transforms.ToTensor(), refu])
        cfg["imCh"] = 1
    elif dname == "TinyImgNet":
        cdat = tinyImgNet
        norm = getnorm(dname)
        # gamma_corr = lambda img : transforms.functional.adjust_gamma(img, gamma=, float=)
        # resize_tr = random.choice([transforms.RandomResizedCrop(56), transforms.CenterCrop(56)])
        # aug_tr = random.choice([transforms.RandomHorizontalFlip(), transforms.RandomAffine(30, translate=10, scale = [0.8])])
        # data_augs = [transforms.RandomResizedCrop(32), transforms.RandomCrop(32), transforms.RandomHorizontalFlip(), transforms.RandomRotation(45)]
        train_trans = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(mean=norm[0], std=norm[1]),
                transforms.RandomResizedCrop(64),
                # transforms.CenterCrop(56),
                # transforms.Resize(32),
                transforms.RandomHorizontalFlip(),
            ]
        )
        val_trans = transforms.Compose(
            [
                transforms.ToTensor(),
                # transforms.CenterCrop(56),
                transforms.Normalize(mean=norm[0], std=norm[1]),
                # transforms.Resize(32)
            ]
        )
        cfg["imCh"] = 3

    ntrain, down = cfg["ntrain"], True
    if dname == "TinyImgNet":
        down = False
        if reconstr == True:
            trainset = cdat(
                root="data/",
                train=True,
                download=down,
                transform=transforms.Compose([transforms.ToTensor()]),
            )
            valset = cdat(
                root="data/",
                train=False,
                download=down,
                transform=transforms.Compose([transforms.ToTensor()]),
            )

        else:
            trainset = cdat(
                root="data/", train=True, download=down, transform=train_trans
            )
            valset = cdat(root="data/", train=False, download=down, transform=val_trans)
        train_loader = DataLoader(
            trainset, batch_size=cfg["batchSize"], shuffle=True, num_workers=4
        )
        val_loader = DataLoader(
            valset, batch_size=cfg["batchSize"], shuffle=False, num_workers=4
        )
        return train_loader, val_loader, None, trainset, valset

    def loadStore(isTrain, ndat):
        nonlocal cdat
        trainset = cdat(root="data/", train=isTrain, download=down, transform=trans)
        train_dataset = torch.utils.data.DataLoader(
            trainset, batch_size=ndat, num_workers=4
        )  # cfg["batchSize"]
        ds = next(iter(train_dataset))
        X, Y = ds[0].clone().numpy(), ds[1].clone().numpy()
        # normA = lambda bx: (bx - np.min(bx)) / (np.max(bx) - np.min(bx) + 1e-10)  # "Normalizing - necessary if no BN"
        # X,Y = normA(X), normA(Y)
        ds = [X, Y]
        print(
            "Data stats",
            cdat,
            X.shape,
            np.mean(X, axis=(0, 2, 3)),
            np.std(X, axis=(0, 2, 3)),
            np.max(X),
            np.min(X),
            " Data should be normalized",
        )
        ds = sklearn.utils.shuffle(*ds)
        return ds[0].astype(np.float16), ds[1].astype(np.int16)

    trX, trY = loadStore(True, ntrain)
    teX, teY = loadStore(False, ntrain // 2)

    def cds(trX, trY, shuffle=True):
        ds = TensorDataset(torch.from_numpy(trX), torch.from_numpy(trY))
        return DataLoader(
            ds, batch_size=cfg["batchSize"], shuffle=shuffle, num_workers=4, pin_memory = True
        )

    return cds(trX, trY), cds(teX, teY, False), None


def tinyImgNet(root: str, train: bool, download: bool, transform) -> Dataset:
    filter_val = train and not download

    if download:
        os.makedirs(root, exist_ok=True)
        r = requests.get(
            "http://cs231n.stanford.edu/tiny-imagenet-200.zip", stream=True
        )
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall(root)

    main_dir_path = os.path.join(root, "tiny-imagenet-200")
    train_path = os.path.join(main_dir_path, "train")
    val_path = os.path.join(main_dir_path, "val")
    if not (os.path.exists(train_path) and os.path.exists(val_path)):
        raise ValueError("Train and Val paths don't exist")

    val_formatted = os.path.join(main_dir_path, "val_formatted")
    if not filter_val and not os.path.exists(val_formatted):
        with open(os.path.join(val_path, "val_annotations.txt"), "r") as f:
            img_class_map = f.readlines()
        for img_class in img_class_map:
            img_name, class_name = img_class.split("\t")[0], img_class.split("\t")[1]
            img_path = os.path.join("images", img_name)
            class_dir = os.path.join(val_formatted, class_name)
            os.makedirs(os.path.join(class_dir, "images"), exist_ok=True)
            copyfile(
                os.path.join(val_path, img_path), os.path.join(class_dir, img_path)
            )
    if train:
        return ImageFolder(train_path, transform)
    else:
        return ImageFolder(val_formatted, transform)


In [3]:


def decay(opt,epoch,optimizerCl):
    if opt[0] == "S" and (epoch + 1) % (opt[1] // 3+opt[1]//10+2 ) == 0:
        for p in optimizerCl.param_groups: p['lr'] *= 0.1
        #print("  D", np.round(optimizerCl.param_groups[0]['lr'],5))

def getAcc(net, dataset,  niter=10000,norm=None):
    correct,total = 0,0
    net.eval()
    with torch.no_grad():
        for cit,data in enumerate(dataset):
            with tca.autocast():
                dsx,dsy = data[0].cuda(),data[1].cuda()
                if len(dsx.size()) == 5:
                    dsx = dsx.squeeze(0)
                    dsy = dsy.squeeze(0)
                total += dsy.size(0)
                outputs = net(dsx.float())
                _, predicted = torch.max(outputs.data, 1)
                correct += torch.eq(predicted, dsy).sum().item()
                if cit>=niter: break
    return correct/total


def getclassifier(cfg,train_dataset,val_dataset,norm=None):

    netCl=ResNet10(n_classes = cfg["ds"][1], n_output_planes = [64, 128, 256, 512], inch = cfg["imCh"]).cuda()
    optimizerCl = optim.SGD(netCl.parameters(), lr=cfg["opt"][2], momentum=0.9, weight_decay=cfg["opt"][3])
    #optimizerCl = optim.SGD(netCl.parameters(), lr=cfg["opt"][2], momentum=0.9, weight_decay=cfg["opt"][3])
    closs,teaccs,trep,loss,clr = 0,[],cfg["opt"][1],nn.CrossEntropyLoss(), cfg["opt"][2]
    print("Train Classifier to explain")
    scaler = tca.GradScaler()
    teAccs,trAccs=[],[]
    clAcc = lambda dataset: getAcc(netCl, dataset,  niter=1e10,norm=norm)
    for epoch in tqdm(range(trep)):
        netCl.train()
        for i, data in enumerate(tqdm(train_dataset, position=0, leave=True)):
            with tca.autocast():
                optimizerCl.zero_grad()
                dsx = data[0]
                dsx,dsy = dsx.cuda(),data[1].cuda()
                if len(dsx.size()) == 5:
                    dsx = dsx.squeeze(0)
                    dsy = dsy.squeeze(0)
                output = netCl(dsx.float())
                errD_real = loss(output, dsy.long())
                scaler.scale(errD_real).backward()
                scaler.step(optimizerCl)
                scaler.update()
            closs = 0.97 * closs + 0.03 * errD_real.item() if i > 20 else 0.8 * closs + 0.2 * errD_real.item()
        decay(cfg["opt"],epoch,optimizerCl)
        netCl.eval()
        teAccs.append(clAcc(val_dataset))
        if (epoch % 4 == 0 and epoch<=13) or (epoch % 20==0 and epoch>13):
            print(epoch, np.round(np.array([closs, teAccs[-1], clAcc(train_dataset)]), 5))
    lcfg = {"testAcc": clAcc(val_dataset), "trainAcc": clAcc(train_dataset)}
    netCl.eval()
    return netCl, lcfg

In [4]:
import numpy as np
import torch.cuda.amp as tca
import torch
from torch import nn
import pdb
from tqdm import tqdm
import torchvision.transforms as transforms


class ClaDecNet(nn.Module):
    def __init__(self, cfg, inShape, nFea):
        super(ClaDecNet, self).__init__()
        self.channel_mult = int(64)
        self.expLinLay = len(inShape) == 2  # linear layer...

        dim = 64 if self.expLinLay else 64 // int(inShape[-1])  # dimension of input
        self.input_dim = np.prod(inShape[1:])
        self.inFea = inShape[-2] if not self.expLinLay else 1
        nLay = int(np.round(np.log2(dim) - 2))

        # Use batchnorm or bias? LeakyRelu or Relu   -->  Does not make much of a difference
        # bn,bias = lambda x:  nn.BatchNorm2d(x),False
        # rel =lambda x:  nn.LeakyReLU(0.01)
        bn, bias = lambda x: nn.Identity(), True
        rel = lambda x: nn.ReLU()

        if (
            self.inFea == 1
        ):  # There is no spatial dimension (or it is one) -> use a dense layer as the first layer
            self.useDense = True
            self.fc_output_dim = max(
                self.input_dim, self.channel_mult
            )  # number of input features
            self.fc = nn.Sequential(
                nn.Linear(self.input_dim, self.fc_output_dim), nn.ReLU(True)
            )  # , nn.BatchNorm1d(self.fc_output_dim)
        else:  # The spatial extend is larger one, use a conv layer, otherwise have too many parameters
            self.fc_output_dim = inShape[1]  # number of input features
            self.fc = nn.Sequential(
                nn.Conv2d(inShape[1], inShape[1], 3, stride=1, padding=1, bias=bias),
                nn.ReLU(True),
            )  # , nn.BatchNorm1d(self.fc_output_dim)
            self.useDense = False

        temp_nLay = 0
        if nLay < 0:
            temp_nLay = nLay
            nLay = 0
        
        self.deconv = [
            nn.ConvTranspose2d(
                self.fc_output_dim,
                self.channel_mult * (2 ** nLay),
                4,
                stride=2,
                padding=1,
                bias=bias,
            ),
            bn(self.channel_mult * (2 ** nLay)),
            rel(None),
        ]
        if temp_nLay < 0:
            self.deconv.append(nn.MaxPool2d((2,2), stride=2))
        for j in range(nLay, 0, -1):
            self.deconv += [
                nn.ConvTranspose2d(
                    self.channel_mult * (2 ** j),
                    self.channel_mult * (2 ** (j - 1)),
                    4,
                    stride=2,
                    padding=1,
                    bias=bias,
                ),
                bn(self.channel_mult * (2 ** (j - 1))),
                rel(None),
            ]
        self.deconv.append(
            nn.ConvTranspose2d(
                self.channel_mult * 1, nFea, 4, stride=2, padding=1, bias=True
            )
        )
        if temp_nLay < -1:
            self.deconv.append(nn.MaxPool2d((2,2), stride=2))
        self.deconv = nn.Sequential(*self.deconv)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        if self.useDense:
            x = x.view(-1, self.input_dim)
            x = self.fc(x)
            x = x.view(-1, self.fc_output_dim, self.inFea, self.inFea)
        else:
            x = self.fc(x)
        x = self.deconv(x)
        x = self.sig(
            x
        )  # This sometimes gives better visualization, but you need to take care to standardize inputs as well
        return x


def getClaDec(cfg, netCl, norm, train_dataset):
    alpha = cfg["alpha"]
    closs, cclloss, crloss, teaccs, trep, clloss, clr = (
        0,
        0,
        0,
        [],
        cfg["opt"][1],
        nn.CrossEntropyLoss(),
        cfg["opt"][2],
    )
    print("Train CLaDec")
    scaler = tca.GradScaler()
    d = next(iter(train_dataset))
    netDec = ClaDecNet(cfg, d[2].squeeze(0).shape, cfg["imCh"]).cuda()
    netDec.train()
    netCl.train()
    optimizerCl = torch.optim.Adam(netDec.parameters(), lr=0.0003, weight_decay=1e-5)
    aeloss = nn.MSELoss()
    ulo = (
        lambda closs, totloss, i: 0.97 * closs + 0.03 * totloss.item()
        if epoch > 20
        else 0.8 * closs + 0.2 * totloss.item()
    )
    for epoch in tqdm(range(trep)):
        for i, data in enumerate(train_dataset):
            with tca.autocast():
                optimizerCl.zero_grad()
#                 if cfg["ds"][0] == "TinyImgNet":
#                     dsx, dsy, dsact = (
#                         data[0].squeeze(0).cuda(),
#                         data[1].squeeze(0).cuda(),
#                         data[2].squeeze(0).cuda(),
#                     )
#                 else:
#                     dsx, dsy, dsact = data[0].cuda(), data[1].cuda(), data[2].cuda()
                dsx, dsy, dsact = (
                    data[0].squeeze(0).cuda(),
                    data[1].squeeze(0).cuda(),
                    data[2].squeeze(0).cuda(),
                )

                output = netDec(dsact.float())
                recloss = aeloss(output, dsx)
                if cfg["ds"][0] == "TinyImgNet":
                    norm = getnorm("TinyImgNet")
                    transform = transforms.Compose(
                        [
                            transforms.Normalize(mean=norm[0], std=norm[1]),
                            transforms.RandomResizedCrop(64),
                            transforms.RandomHorizontalFlip(),
                        ]
                    )
                    claloss = clloss(netCl(transform(output)), dsy.long())
                else:
                    claloss = clloss(netCl(output), dsy.long())
                totloss = (1 - alpha) * recloss + alpha * claloss
                scaler.scale(totloss).backward()
                scaler.step(optimizerCl)
                scaler.update()
                closs, cclloss, crloss = (
                    ulo(closs, totloss, epoch),
                    ulo(cclloss, claloss, epoch),
                    ulo(crloss, recloss, epoch),
                )

        decay(cfg["opt"], epoch, optimizerCl)
        if (epoch % 2 == 0 and epoch <= 10) or (epoch % 10 == 0 and epoch > 10):
            print(epoch, np.round(np.array([closs, crloss, cclloss]), 5))

    lcfg = {"ClaLo": closs}
    netDec.eval()
    return netDec, lcfg


def getActModel(cfg, classifier):
    ind = cfg["layInd"]
    if cfg["ds"][0] == "TinyImgNet":
        if ind < -1:
            ind = ind - 3
    actModel = nn.Sequential(*list(classifier.children())[:ind])
    actModel.eval()
    return actModel


class RefAE(nn.Module):
    def __init__(self, cfg, inShape):
        super(RefAE, self).__init__()
        self.cladec = ClaDecNet(cfg, inShape, cfg["imCh"])
        self.cladec.train()
        cla = ResNet10(n_classes = cfg["ds"][1], n_output_planes = [64, 128, 256, 512], inch = cfg["imCh"]).cuda()
        actModel = getActModel(cfg, cla)
        actModel.train()
        self.seq = nn.Sequential(actModel, self.cladec)

    def forward(self, x):
        return self.seq(x)


def getRefAE(cfg, trds, train_dataset):
    closs, teaccs, trep, clloss, clr = (
        0,
        [],
        cfg["opt"][1],
        nn.CrossEntropyLoss(),
        cfg["opt"][2],
    )
    print("Train RefAE")
    scaler = tca.GradScaler()
    d = next(iter(trds))
    netDec = RefAE(cfg, d[2].squeeze(0).shape).cuda()
    netDec.train()
    optimizerCl = torch.optim.Adam(netDec.parameters(), lr=0.0003, weight_decay=1e-5)
    aeloss = nn.MSELoss()
    ulo = (
        lambda closs, totloss, i: 0.97 * closs + 0.03 * totloss.item()
        if epoch > 20
        else 0.8 * closs + 0.2 * totloss.item()
    )
    for epoch in tqdm(range(trep)):
        for i, data in enumerate(train_dataset):
            with tca.autocast():
                optimizerCl.zero_grad()
#                 if cfg["ds"][0] == "TinyImgNet":
#                     dsx = data[0].squeeze(0).cuda()
#                 else:
#                     dsx = data[0].cuda()
                dsx = data[0].squeeze(0).cuda()
                output = netDec(dsx.float())
                recloss = aeloss(output, dsx)
                scaler.scale(recloss).backward()
                scaler.step(optimizerCl)
                scaler.update()
                closs = ulo(closs, recloss, epoch)

        decay(cfg["opt"], epoch, optimizerCl)
        if (epoch % 2 == 0 and epoch <= 10) or (epoch % 10 == 0 and epoch > 10):
            print(epoch, np.round(np.array([closs]), 5))

    lcfg = {"RefAELo": closs}
    netDec.eval()
    return netDec, lcfg


In [5]:
import torch
from torch.utils.data import Dataset,TensorDataset
import numpy as np,os
import torch.cuda.amp as tca
from matplotlib import pyplot as plt

def showPics(pics, mtit="", tits =None,nrows = 16,ncols=12,fname="expPics"):
    picPerFig = nrows * ncols
    for i in range(max(1,pics.shape[0] // picPerFig)):
        fig = plt.figure(figsize=(20,30))
        wm = plt.get_current_fig_manager()
        wm.resize(*wm.window.maxsize())  # wm.window.state('zoomed')
        fig.suptitle(mtit, fontsize=8)
        for j in range(picPerFig):
            if i * picPerFig + j == pics.shape[0]: break
            ax1 = plt.subplot(nrows, (picPerFig - 1) // nrows + 1, j + 1)
            if not tits is None and i*picPerFig + j<len(tits):
               tit=ax1.set_title(str(tits[i*picPerFig + j]), fontsize=16)
               plt.setp(tit, color='black')
            cpic=pics[i * picPerFig + j].squeeze()
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
            ax1.axis('off')
            cpic = 1-(cpic-np.min(cpic))/(np.max(cpic)-np.min(cpic)+1e-10)
            plt.imshow(cpic.astype(np.float32),cmap='Greys')
        plt.subplots_adjust(wspace=0, hspace=0)
        plt.savefig(fname+str(i)+ ".png")
        plt.close()
        if i * picPerFig + j >= pics.shape[0] - 1: break

def getActs(ds,actModel,cfg):
        acts=[]
        X,y=[],[]
        for i, data in enumerate(ds):
            with tca.autocast():
                dsx, dsy = data[0].cuda(), data[1].cuda()
                X.append(data[0])
                y.append(data[1])
                #classifier(dsx)
                acts.append(actModel(dsx).detach().cpu())
        X=torch.cat(X,dim=0)
        y=torch.cat(y, dim=0)
        conacts=torch.cat(acts,dim=0)
        dsact=TensorDataset(X,y,conacts)
        return torch.utils.data.DataLoader(dsact, batch_size=cfg["batchSize"], shuffle=True, num_workers=4)


In [6]:
dummy=False
#dummy = True
#cfg={ 'ds': ('Fash', 10),  #Dataset either  ('Ci100', 100) or ('Ci10', 10)
#      'batchSize': 128, 'opt': ('S', 1 if dummy else 64, 0.1, 0.0001), #optimizer settings
#      'layInd':-1, #Layer to explain (from last layer back, ie. -1 is last (linear), -2 is last conv, -3 second last conv)
#      'alpha': 0.001, #tradeoff parameter reconstruction vs. classification loss
#      'ntrain': 500 if dummy else 60000}
cfg={ 'ds': ('MNIST', 10),  #Dataset either  ('Ci100', 100) or ('Ci10', 10)
    'batchSize': 128, 'opt': ('S', 1 if dummy else 60, 0.1, 0.0001), #optimizer settings
    'layInd':-1, #Layer to explain (from last layer back, ie. -1 is last (linear), -2 is last conv, -3 second last conv)
    'alpha': 0.001, #tradeoff parameter reconstruction vs. classification loss
    'ntrain': 500 if dummy else 60000}
print("Executing config",cfg)
cfg["num_classes"]=cfg["ds"][1]
#Get Data
print("Get dataset")
train_dataset, val_dataset,norm=getFullDS(cfg)


Executing config {'ds': ('MNIST', 10), 'batchSize': 128, 'opt': ('S', 60, 0.1, 0.0001), 'layInd': -1, 'alpha': 0.001, 'ntrain': 60000}
Get dataset
Data stats <class 'torchvision.datasets.mnist.MNIST'> (60000, 1, 64, 64) [0.1308931] [0.30838418] 1.0 0.0  Data should be normalized
Data stats <class 'torchvision.datasets.mnist.MNIST'> (10000, 1, 64, 64) [0.13275763] [0.31077635] 1.0 0.0  Data should be normalized


In [7]:
model_path = "trained_models/resnet/MNIST"
import os
os.makedirs(model_path, exist_ok=True)

1

In [ ]:
%%time
# Train and save non-reflective Model
cfg["opt"] = ('S', 1 if dummy else 20, 0.1, 0.0001)
classifier, lcfg = getclassifier(cfg,  train_dataset, val_dataset, norm=norm)
print("Classifier Accuracy",lcfg)
torch.save(classifier.state_dict(), os.path.join(model_path, "classifier.pt"))

In [8]:
classifier = ResNet10(n_classes = cfg["ds"][1], n_output_planes = [64, 128, 256, 512], inch = cfg["imCh"])
classifier.load_state_dict(torch.load(os.path.join(model_path, "classifier.pt")))
classifier.cuda()

ResNet(
  (layer0): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): BasicBlock(
      (block): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (block): Sequential(
        (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): 

In [9]:
def getReconstrDS(acts_ds,refAE,cladecNet,cfg):
    X_cladec,X_refae,y=[],[],[]
    cladec_loss, refae_loss = 0, 0
    for i, data in enumerate(acts_ds):
        #if i > 2: break
        with tca.autocast():
            with torch.no_grad():
                dsx, dsy,dsact = data[0].cuda(), data[1], data[2].cuda()
                outCla = cladecNet(dsact).detach().cpu().float()
                outAE = refAE(dsx).detach().cpu().float()
                dsx = dsx.detach().squeeze().cpu().float()
                cladec_loss += (outCla - dsx).pow(2).mean()
                refae_loss += (outAE - dsx).pow(2).mean()
                X_cladec.append(outCla)
                X_refae.append(outAE)
                y.append(dsy)
    
    
    refae_loss = refae_loss/i
    cladec_loss = cladec_loss/i
    X_cladec = torch.cat(X_cladec, dim = 0)
    X_refae = torch.cat(X_refae, dim = 0)
    y = torch.cat(y, dim=0)
    cladec_ds = TensorDataset(X_cladec, y)
    refae_ds = TensorDataset(X_refae, y)
    
    cladec_loader = torch.utils.data.DataLoader(cladec_ds, batch_size=cfg["batchSize"], shuffle=True, num_workers=4)
    refae_loader = torch.utils.data.DataLoader(refae_ds, batch_size=cfg["batchSize"], shuffle=True, num_workers=4)
    return cladec_loader, refae_loader, cladec_loss, refae_loss        
        #X=torch.cat(X,dim=0)
        #y=torch.cat(y, dim=0)
        #dsact=TensorDataset(X,y,conacts)
        #return torch.utils.data.DataLoader(dsact, batch_size=cfg["batchSize"], shuffle=True, num_workers=4)


In [ ]:
batch_ordered = torch.load(os.path.join(model_path, "original.pt"))
clf_preds = torch.load(os.path.join(model_path,"clf_preds.pt"))


In [ ]:
batch_ordered = [x for x in val_dataset]
torch.save(batch_ordered,os.path.join(model_path, "original.pt"))
clf_preds = []
for (x,y) in batch_ordered:
    with torch.no_grad():
        with tca.autocast():
            clf_preds.append(classifier(x.cuda()).argmax(dim=1).detach().cpu())
torch.save(clf_preds, os.path.join(model_path,"clf_preds.pt"))


In [9]:
import torch
from torch.utils.data import TensorDataset, Dataset, IterableDataset
import models as clModel,dutils
import cladec
import numpy as np,os
import torch.cuda.amp as tca
from matplotlib import pyplot as plt
import pdb
from tqdm import tqdm
class LargeActDataset(IterableDataset):
    def __init__(self, orig_loader, act_model, device):
        super(LargeActDataset, self).__init__()
        self.orig_loader = orig_loader
        self.act_model = act_model
        self.device = device


    def __iter__(self) :
        for (img, label) in self.orig_loader:
            with torch.no_grad():
                with tca.autocast():
                    act = self.act_model(img.to(self.device)).detach().cpu()
            yield (img.detach().cpu(), label, act)


def getActs(ds,actModel,cfg):
    dsact = LargeActDataset(ds, actModel, cfg["device"])
    return torch.utils.data.DataLoader(dsact, batch_size=1)


In [11]:
j = 0
for daat in tqdm(trds):
    j+=1
    if j > 50:
        break

 11%|█         | 50/469 [00:01<00:12, 34.40it/s]


In [10]:
cfg["device"] = torch.device("cuda:0")

In [ ]:
ae_train_loss_list = []
ae_val_loss_list = []
cladec_train_loss_list = []
cladec_val_loss_list = []
cladec_val_acc = []
refae_val_acc = []

for i in range(4, 5):
    cfg["opt"] = ('S', 1 if dummy else 60, 0.1, 0.0001)
    refae_path = os.path.join(model_path, "refAE_{}.pt".format(str(i)))
    cladec_path = os.path.join(model_path, "cladecNet_{}.pt".format(str(i)))
    cfg["layInd"] = -1*i
    #get Activations
    actModel = getActModel(cfg, classifier)
    actModel.eval()
    trds=getActs(train_dataset, actModel,cfg)
    
    if i > 3:
        #get RefAE
        refAE,rcfg =getRefAE(cfg,trds, train_dataset) #Does not use activations themselves, only needs shape
        print(rcfg)
        torch.save(refAE.state_dict(), refae_path)
    # Train ClaDec
    cladecNet,ccfg=  getClaDec(cfg,classifier,None,trds)
    print("ClaDec Final loss", ccfg)
    torch.save(cladecNet.state_dict(), cladec_path)
    
    # d = next(iter(trds))
    # refAE = RefAE(cfg, d[2].squeeze(0).shape).cuda()
    # refAE.load_state_dict(torch.load(refae_path))
    
    # cladecNet = ClaDecNet(cfg, d[2].squeeze(0).shape, cfg["imCh"]).cuda()
    # refAE.eval()
    # cladecNet.eval()
    # actModel.eval()
    
    # train_acts = getActs(train_dataset, actModel, cfg)
    # val_acts = getActs(val_dataset, actModel, cfg)
    
    # cladec_train, refae_train, cladec_train_loss, refae_train_loss = getReconstrDS(train_acts, refAE, cladecNet, cfg)
    # cladec_val, refae_val, cladec_val_loss, refae_val_loss = getReconstrDS(train_acts, refAE, cladecNet, cfg)
    
    # ae_train_loss_list.append(refae_train_loss)
    # ae_val_loss_list.append(refae_val_loss)
    # cladec_train_loss_list.append(cladec_train_loss)
    # cladec_val_loss_list.append(cladec_val_loss)
    

    # cfg["opt"] = ('S', 1 if dummy else 20, 0.1, 0.0001)
    # clf_cladec, lcfg_cladec = getclassifier(cfg,  cladec_train, cladec_val, norm=norm)
    # cladec_val_acc.append(lcfg_cladec["testAcc"])
    # torch.save(clf_cladec.state_dict(), os.path.join(model_path, "clf_cladec_" + str(i) + ".pt"))
    # clf_ae, lcfg_ae = getclassifier(cfg,  refae_train, refae_val, norm=norm)
    # refae_val_acc.append(lcfg_ae["testAcc"])
    # torch.save(clf_ae.state_dict(), os.path.join(model_path, "clf_ae_" + str(i) + ".pt"))
    # torch.save({"cladec_train_loss" : cladec_train_loss_list, "cladec_val_loss": cladec_val_loss_list, "cladec_val_acc" : cladec_val_acc}, os.path.join(model_path,"cladec_metrics_{}.pt".format(str(i))))
    # torch.save({"ae_train_loss" : ae_train_loss_list, "ae_val_loss": ae_val_loss_list, "ae_val_acc" : refae_val_acc}, os.path.join(model_path, "ae_metrics_{}.pt".format(str(i))))



    # ae_reconstr = []
    # cladec_reconstr = []
    # with tca.autocast():
    #     for (x,y) in (batch_ordered):
    #         y_pred_ae = clf_ae(x.cuda()).argmax(dim=1).detach().cpu()
    #         ae_reconstr.append((refAE(x.cuda()).detach().cpu(),y_pred_ae))
    #         y_pred_cladec = clf_cladec(x.cuda()).argmax(dim=1).detach().cpu()
    #         cladec_reconstr.append((cladecNet(actModel(x.cuda())).detach().cpu(),y_pred_cladec))
    
    # torch.save(ae_reconstr,os.path.join(model_path, "ae_reconstr_" + str(i)+ ".pt"))
    # torch.save(cladec_reconstr,os.path.join(model_path, "cladec_reconstr_" + str(i)+ ".pt"))

Train RefAE


  2%|▏         | 1/60 [01:02<1:01:41, 62.73s/it]

0 [0.00018]


  5%|▌         | 3/60 [03:07<59:15, 62.37s/it]  

2 [0.0001]


...

 18%|█▊        | 11/60 [26:11<1:56:40, 142.87s/it]

10 [7.0e-05 7.0e-05 7.1e-04]


 35%|███▌      | 21/60 [50:03<1:32:58, 143.04s/it]

20 [6.00e-05 5.00e-05 1.08e-03]


 38%|███▊      | 23/60 [54:50<1:28:19, 143.23s/it]

Sequential(
  (0): BasicBlock(
    (block): Sequential(
      (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)

In [ ]:
actModel = getActModel(cfg, classifier)
actModel.eval()
trds=getActs(train_dataset,actModel,cfg)

In [ ]:
refAE,rcfg =getRefAE(cfg,trds)
print(rcfg)

Train RefAE



 50%|█████     | 1/2 [00:19<00:19, 19.14s/it]

0 [0.00115]



100%|██████████| 2/2 [00:38<00:00, 19.17s/it]

{'RefAELo': 0.0006595195792957057}


In [ ]:
cladecNet,ccfg=  getClaDec(cfg,classifier,norm,trds)

Train CLaDec



 50%|█████     | 1/2 [00:19<00:19, 19.49s/it]

0 [0.00438 0.00433 0.05723]



100%|██████████| 2/2 [00:39<00:00, 19.56s/it]


In [25]:
!pip install skimage

    ERROR: Command errored out with exit status 1:
     command: /home/harshv834/my-envs/ml_def/bin/python -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-khfm7_zb/skimage_fd8c4a1ba1f548ad9d5356e9ee2a06df/setup.py'"'"'; __file__='"'"'/tmp/pip-install-khfm7_zb/skimage_fd8c4a1ba1f548ad9d5356e9ee2a06df/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /tmp/pip-pip-egg-info-5w1e6jf5
         cwd: /tmp/pip-install-khfm7_zb/skimage_fd8c4a1ba1f548ad9d5356e9ee2a06df/
    Complete output (3 lines):
    
    *** Please install the `scikit-image` package (instead of `skimage`) ***
    
    ----------------------------------------
ERROR: Could not find a version that satisfies the requirement skimage (from versions: 0.0)
ERRO

In [66]:
import torch
import numpy as np
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from PIL import Image
import imageio

def save_reconstructions(cfg, data_loader, model_path, model, train=True, cladec=True):
    rec_path = os.path.join(model_path, "rec_cladec" if cladec else "rec_refae", "reconstructions_{}".format(str(-1*cfg["layInd"])))
    os.makedirs(rec_path, exist_ok = True)
    dset_path = os.path.join(rec_path, "train" if train else "val")
    os.makedirs(dset_path, exist_ok=True)
    img_tr = transforms.ToPILImage()
    total_classes = set()
    idx = 0
    for (img, label) in tqdm(data_loader, position=0, leave=True):
        label = np.array(label)
        if len(total_classes) < cfg["ds"][1]:
            label_classes = set(label)
            new_classes = label_classes.difference(total_classes)
            total_classes = total_classes.union(label_classes)
            for i in new_classes:
                class_path = os.path.join(dset_path, str(i))
                os.makedirs(class_path, exist_ok=True)
        
        with tca.autocast():
            act = model(img.to(cfg["device"])).detach().cpu()
        for j in range(len(img)):
            idx +=1
            vec = act[j].squeeze(0).numpy()
#             vec = img_tr(act[j])
            vec_label = label[j]
            vec_path = os.path.join(dset_path, str(vec_label),str(idx) + ".jpg")
            imageio.imwrite(vec_path,(vec*255.9).astype(np.uint8))

In [12]:
cfg["device"] = torch.device("cuda:0")

In [85]:
for i in tqdm(range(2,3)):
    cfg["layInd"] = -1*i
    actModel = getActModel(cfg, classifier)
    actModel.eval()
    trds=getActs(train_dataset,actModel,cfg)

    d = next(iter(trds))
    refAE = RefAE(cfg, d[2].squeeze(0).shape).cuda()
    refAE.load_state_dict(torch.load(os.path.join(model_path, "refAE_"+  str(i) + ".pt")))
    cladecNet = ClaDecNet(cfg, d[2].squeeze(0).shape, cfg["imCh"]).cuda()
    cladecNet.load_state_dict(torch.load(os.path.join(model_path, "cladecNet_"+  str(i) + ".pt")))
    refAE.eval()
    cladecNet.eval()
    cladec_ae = torch.nn.Sequential(actModel, cladecNet)
    cladec_ae.eval()
    save_reconstructions(cfg, train_dataset, model_path, refAE, train=True, cladec=False)
    save_reconstructions(cfg, val_dataset, model_path, refAE, train=False, cladec=False)
    save_reconstructions(cfg, train_dataset, model_path, cladec_ae, train=True, cladec=True)
    save_reconstructions(cfg, val_dataset, model_path, cladec_ae, train=False, cladec=True)

100%|██████████| 1/1 [01:53<00:00, 113.77s/it]


In [13]:
import torch
from torch.utils.data import TensorDataset, Dataset, IterableDataset
import numpy as np,os
import torch.cuda.amp as tca
from matplotlib import pyplot as plt
import pdb
from tqdm import tqdm
class LargeActDataset(IterableDataset):
    def __init__(self, orig_loader, act_model, device):
        super(LargeActDataset, self).__init__()
        self.orig_loader = orig_loader
        self.act_model = act_model
        self.device = device


    def __iter__(self) :
        for (img, label) in self.orig_loader:
            with torch.no_grad():
                with tca.autocast():
                    act = self.act_model(img.to(self.device)).detach().cpu()
            yield (img.detach().cpu(), label, act)

def getActs(ds,actModel,cfg):
    dsact = LargeActDataset(ds, actModel, cfg["device"])
    return torch.utils.data.DataLoader(dsact, 
                                       batch_size=1)
import torchvision.transforms as transforms

class LargeReconstrDataset(IterableDataset):
    def __init__(self, orig_loader, model, device, train):
        super(LargeReconstrDataset, self).__init__()
        self.orig_loader = orig_loader
        self.model = model
        self.device = device
        norm = getnorm("TinyImgNet")
        tr_list = [transforms.Normalize(mean = norm[0], std=norm[1])]
        if train:
            tr_list.append(transforms.RandomResizedCrop(64))
            tr_list.append(transforms.RandomHorizontalFlip())
            
        self.transform = transforms.Compose(tr_list)

    def __iter__(self) :
        for (img, label) in self.orig_loader:
            with torch.no_grad():
                with tca.autocast():
                    reconstr = self.model(img.to(self.device)).detach().cpu()
                reconstr = self.transform(reconstr)
            yield (reconstr, label)

def getReconstr(ds,model,cfg, train):
    dsact = LargeReconstrDataset(ds, model, cfg["device"], train)
    return torch.utils.data.DataLoader(dsact, batch_size=1)

def getReconstrDS(acts_ds,refAE,cladecNet):
    cladec_loss, refae_loss = 0, 0
    for i, data in enumerate(tqdm(acts_ds, position=0, leave=True)):
        #if i > 2: break
        with tca.autocast():
            with torch.no_grad():
                dsx, dsact = data[0].cuda(), data[2].cuda()
                if len(dsx.size()) == 5:
                    dsx = dsx.squeeze(0)
                    dsact = dsact.squeeze(0)
                outCla = cladecNet(dsact).detach().cpu().float()
                outAE = refAE(dsx).detach().cpu().float()
                dsx = dsx.detach().squeeze().cpu().float()
                cladec_loss += (outCla - dsx).pow(2).mean()
                refae_loss += (outAE - dsx).pow(2).mean()
    
    
    refae_loss = refae_loss/i
    cladec_loss = cladec_loss/i
    return cladec_loss, refae_loss
        #X=torch.cat(X,dim=0)
        #y=torch.cat(y, dim=0)
        #dsact=TensorDataset(X,y,conacts)
        #return torch.utils.data.DataLoader(dsact, batch_size=cfg["batchSize"], shuffle=True, num_workers=4)



In [79]:
from PIL import Image
import os
import os.path
from typing import Any, Callable, cast, Dict, List, Optional, Tuple
from torchvision.datasets import DatasetFolder
def bw_loader(path):
    img = Image.open(path)
    return img
IMG_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')

class BWFolder(DatasetFolder):
    """A generic data loader where the images are arranged in this way by default: ::
        root/dog/xxx.png
        root/dog/xxy.png
        root/dog/[...]/xxz.png
        root/cat/123.png
        root/cat/nsdf3.png
        root/cat/[...]/asd932_.png
    This class inherits from :class:`~torchvision.datasets.DatasetFolder` so
    the same methods can be overridden to customize the dataset.
    Args:
        root (string): Root directory path.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        loader (callable, optional): A function to load an image given its path.
        is_valid_file (callable, optional): A function that takes path of an Image file
            and check if the file is a valid file (used to check of corrupt files)
     Attributes:
        classes (list): List of the class names sorted alphabetically.
        class_to_idx (dict): Dict with items (class_name, class_index).
        imgs (list): List of (image path, class_index) tuples
    """

    def __init__(
            self,
            root: str,
            transform: Optional[Callable] = None,
            target_transform: Optional[Callable] = None,
            loader: Callable[[str], Any] = bw_loader,
            is_valid_file: Optional[Callable[[str], bool]] = None,
    ):
        super(BWFolder, self).__init__(root, loader, IMG_EXTENSIONS if is_valid_file is None else None,
                                          transform=transform,
                                          target_transform=target_transform,
                                          is_valid_file=is_valid_file)
        self.imgs = self.samples

In [82]:
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

def getReconstr(cfg, model_path, cladec=True, train=True):
    rec_path = os.path.join(model_path, "rec_cladec" if cladec else "rec_refae", "reconstructions_{}".format(str(-1*cfg["layInd"])),"train" if train else "val")
    if train:
        transform = transforms.Compose([transforms.ToTensor(), transforms.RandomResizedCrop(64), transforms.RandomHorizontalFlip()])
    else:
        transform = transforms.ToTensor()
    ds_reconstr = BWFolder(rec_path, transform = transform)
    return torch.utils.data.DataLoader(ds_reconstr, batch_size=cfg["batchSize"], shuffle=train, num_workers = 4, pin_memory=True)


In [45]:
(img, label) = next(iter(val_dataset))

In [49]:
one = img[0].numpy()

In [51]:
import matplotlib.pyplot as plt

In [56]:
print(one.min())

0.0


In [58]:
import imageio
imageio.imwrite("one.jpg", (one.reshape(64,64)*255.9).astype(np.uint8))

In [59]:
from PIL import Image
img = Image.open("one.jpg")

torch.Size([1, 64, 64])


In [91]:
cfg["opt"] = ('S', 1 if dummy else 20, 0.1, 0.0001)
cfg["device"] = torch.device("cuda:0")
ae_train_loss_list = []
ae_val_loss_list = []
cladec_train_loss_list = []
cladec_val_loss_list = []
cladec_val_acc = []
refae_val_acc = []

# for i in range(4, 5):
#     refae_path = os.path.join(model_path, "refAE_{}.pt".format(str(i)))
#     cladec_path = os.path.join(model_path, "cladecNet_{}.pt".format(str(i)))
#     cfg["layInd"] = -1*i
#     #get Activations
#     actModel = getActModel(cfg, classifier)
#     actModel.eval()
#     trds=getActs(train_loader,actModel,cfg)





for i in tqdm(range(2,5)):
    cfg["layInd"] = -1*i
    actModel = getActModel(cfg, classifier)
    actModel.eval()
    trds=getActs(train_dataset,actModel,cfg)

    d = next(iter(trds))
    refAE = RefAE(cfg, d[2].squeeze(0).shape).cuda()
    refAE.load_state_dict(torch.load(os.path.join(model_path, "refAE_"+  str(i) + ".pt")))
    cladecNet = ClaDecNet(cfg, d[2].squeeze(0).shape, cfg["imCh"]).cuda()

    cladecNet.load_state_dict(torch.load(os.path.join(model_path, "cladecNet_"+  str(i) + ".pt")))
    refAE.eval()
    cladecNet.eval()
    cladec_ae = torch.nn.Sequential(actModel, cladecNet)
    cladec_ae.eval()
    
    train_acts = getActs(train_dataset, actModel, cfg)
    val_acts = getActs(val_dataset, actModel, cfg)
    
    #cladec_train = getReconstr(cfg, model_path, cladec=True, train=True)
    #refae_train = getReconstr(cfg, model_path, cladec=False, train=True)
    cladec_train_loss, refae_train_loss = getReconstrDS(train_acts, refAE, cladecNet)
    #cladec_val= getReconstr(cfg, model_path, cladec=True, train=False)
    #refae_val = getReconstr(cfg, model_path, cladec=False, train=False)
    cladec_val_loss, refae_val_loss = getReconstrDS(train_acts, refAE, cladecNet)
    
    ae_train_loss_list.append(refae_train_loss)
    ae_val_loss_list.append(refae_val_loss)
    cladec_train_loss_list.append(cladec_train_loss)
    cladec_val_loss_list.append(cladec_val_loss)
    print(ae_train_loss_list)
    print(ae_val_loss_list)
    print(cladec_train_loss_list)
    print(cladec_val_loss_list)

#     if i >1:    
#         clf_cladec, lcfg_cladec = getclassifier(cfg,  cladec_train, cladec_val, norm=None)
#         cladec_val_acc.append(lcfg_cladec["testAcc"])
#         torch.save(clf_cladec.state_dict(),os.path.join(model_path, "clf_cladec_" + str(i) + ".pt"))
#         torch.save({"cladec_train_loss" : cladec_train_loss_list, "cladec_val_loss": cladec_val_loss_list, "cladec_val_acc" : cladec_val_acc}, os.path.join(model_path,"cladec_metrics_{}.pt".format(str(i))))

    
#     clf_ae, lcfg_ae = getclassifier(cfg,  refae_train, refae_val, norm=None)
#     refae_val_acc.append(lcfg_ae["testAcc"])
#     torch.save(clf_ae.state_dict(),os.path.join(model_path, "clf_ae_" + str(i) + ".pt"))
#     torch.save({"ae_train_loss" : ae_train_loss_list, "ae_val_loss": ae_val_loss_list, "ae_val_acc" : refae_val_acc}, os.path.join(model_path, "ae_metrics_{}.pt".format(str(i))))

print("cladec metrics")
print(cladec_train_loss)
print("refae metrics")
print(refae_val_loss)


#     refAE,rcfg = getRefAE(cfg,trds, train_loader) #Does not use activations themselves, only needs shape
#     print(rcfg)
#     torch.save(refAE.state_dict(), refae_path)

#     # Train ClaDec
#     cladecNet,ccfg=  getClaDec(cfg,classifier,None,trds)
#     print("ClaDec Final loss", ccfg)
#     torch.save(cladecNet.state_dict(), cladec_path)

    # refAE.eval()
    # cladecNet.eval()
    # actModel.eval()
    
    # train_acts = getActs(train_loader, actModel, cfg)
    # val_acts = getActs(val_loader, actModel, cfg)
    
    # cladec_train = getReconstr(train_loader, torch.nn.Sequential(actModel, cladecNet),cfg, train=True)
    # refae_train = getReconstr(train_loader, refAE,cfg, train=True)
    # cladec_train_loss, refae_train_loss = getReconstrDS(train_acts, refAE, cladecNet)
    # cladec_val= getReconstr(val_loader, torch.nn.Sequential(actModel, cladecNet),cfg, train=False)
    # refae_val = getReconstr(val_loader, refAE,cfg, train=False)
    # cladec_val_loss, refae_val_loss = getReconstrDS(train_acts, refAE, cladecNet)
    
    # ae_train_loss_list.append(refae_train_loss)
    # ae_val_loss_list.append(refae_val_loss)
    # cladec_train_loss_list.append(cladec_train_loss)
    # cladec_val_loss_list.append(cladec_val_loss)

    
    # clf_cladec, lcfg_cladec = getclassifier(cfg,  cladec_train, cladec_val, norm=None)
    # cladec_val_acc.append(lcfg_cladec["testAcc"])
    # torch.save(clf_cladec.state_dict(),os.path.join(model_path,"clf_cladec_" + str(i) + ".pt"))
    # clf_ae, lcfg_ae = getclassifier(cfg,  refae_train, refae_val, norm=None)
    # refae_val_acc.append(lcfg_ae["testAcc"])
    # torch.save(clf_ae.state_dict(),os.path.join(model_path,"clf_ae_" + str(i) + ".pt"))
    # torch.save({"cladec_train_loss" : cladec_train_loss_list, "cladec_val_loss": cladec_val_loss_list, "cladec_val_acc" : cladec_val_acc}, os.path.join(model_path,"cladec_metrics_{}.pt".format(str(i))))
    # torch.save({"ae_train_loss" : ae_train_loss_list, "ae_val_loss": ae_val_loss_list, "ae_val_acc" : refae_val_acc}, os.path.join(model_path,"ae_metrics_{}.pt".format(str(i))))





469it [01:49,  4.27it/s]:00<?, ?it/s]
469it [01:48,  4.32it/s]
 33%|███▎      | 1/3 [03:38<07:17, 218.73s/it]

[tensor(0.1327)]
[tensor(0.1327)]
[tensor(0.1313)]
[tensor(0.1312)]


469it [01:53,  4.13it/s]
469it [01:53,  4.12it/s]
 67%|██████▋   | 2/3 [07:26<03:44, 224.03s/it]

[tensor(0.1327), tensor(0.1331)]
[tensor(0.1327), tensor(0.1331)]
[tensor(0.1313), tensor(0.1323)]
[tensor(0.1312), tensor(0.1323)]


469it [02:29,  3.13it/s]
469it [02:28,  3.15it/s]
100%|██████████| 3/3 [12:25<00:00, 248.51s/it]

[tensor(0.1327), tensor(0.1331), tensor(0.1333)]
[tensor(0.1327), tensor(0.1331), tensor(0.1333)]
[tensor(0.1313), tensor(0.1323), tensor(0.1329)]
[tensor(0.1312), tensor(0.1323), tensor(0.1329)]
cladec metrics
tensor(0.1329)
refae metrics
tensor(0.1333)


In [70]:
(dat, img) = next(iter(cladec_train))

In [69]:
cladec_train = getReconstr(cfg, model_path, cladec=True, train=True)

In [88]:
print(ae_train_loss_list)
print(ae_val_loss_list)
print(cladec_train_loss_list)
print(cladec_val_loss_list)

[]
[]
[]
[]


In [95]:
print(getAcc(classifier, val_dataset))

0.9949
